In [38]:
import pandas as pd
import numpy as np
import turicreate as tc
import copy
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from os import path
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from datetime import datetime, timedelta
from tqdm.auto import tqdm
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500
tqdm.pandas()

In [75]:
transactions = pd.read_csv("./selected_transactions.csv")
articles = pd.read_csv("./selected_articles.csv")
customers = pd.read_csv("./selected_customers.csv")
transactions['customer_id'] = transactions['customer_id'].astype('int')
transactions['article_id'] = transactions['article_id'].astype('int')
customers["customer_id"] = customers["customer_id"].astype("int")
articles['article_id'] = articles['article_id'].astype('int')
actions = pd.DataFrame(transactions).merge(articles, how='left', on='article_id')

In [ ]:
target = 'purchase_count'
user_id = 'customer_id'
item_id = 'article_id'
actions_sf = tc.SFrame(actions)
action_train, action_test = tc.recommender.util.random_split_by_user(actions_sf, 
                                                                     user_id=user_id,
                                                                     item_id=item_id,
                                                                     random_seed=0)

## 4. Data Modelling
### 4.1 Method 1: Content-based Recommendation System
Pairwise item-item Similarities - easily run out of memory！ (Method can only be used for small/low-dimensional data）

In [4]:
articles_profiles = articles.drop(columns='article_id')
articles_profiles

,Feature 0,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Feature 11,Feature 12,Feature 13,Feature 14,Feature 15,Feature 16,Feature 17,Feature 18,Feature 19,Feature 20,Feature 21,Feature 22,Feature 23,Feature 24,Feature 25,Feature 26,Feature 27,Feature 28,Feature 29,Feature 30,Feature 31,Feature 32,Feature 33,Feature 34,Feature 35,Feature 36,Feature 37,Feature 38,Feature 39,Feature 40,Feature 41,Feature 42,Feature 43,Feature 44,Feature 45,Feature 46,Feature 47,Feature 48,Feature 49,Feature 50,Feature 51,Feature 52,Feature 53,Feature 54,Feature 55,Feature 56,Feature 57,Feature 58,Feature 59,Feature 60,Feature 61,Feature 62,Feature 63,Feature 64,Feature 65,Feature 66,Feature 67,Feature 68,Feature 69,Feature 70,Feature 71,Feature 72,Feature 73,Feature 74,Feature 75,Feature 76,Feature 77,Feature 78,Feature 79,Feature 80,Feature 81,Feature 82,Feature 83,Feature 84,Feature 85,Feature 86,Feature 87,Feature 88,Feature 89,Feature 90,Feature 91,Feature 92,Feature 93,Feature 94,Feature 95,Feature 96,Feature 97,Feature 98,Feature 99,Feature 100,Feature 101,Feature 102,Feature 103,Feature 104,Feature 105,Feature 106,Feature 107,Feature 108,Feature 109,Feature 110,Feature 111,Feature 112,Feature 113,Feature 114,Feature 115,Feature 116,Feature 117,Feature 118,Feature 119,Feature 120,Feature 121,Feature 122,Feature 123,Feature 124,Feature 125,Feature 126,Feature 127,Feature 128
0,0.408531,-0.966922,0.499474,0.365774,0.294214,0.055154,-0.066833,-0.113461,-0.184069,-0.175194,0.210305,-0.261333,-0.262689,0.111734,-0.196819,0.418919,-0.550053,0.340710,0.042336,-0.205734,-0.061570,-0.046016,-0.131113,0.251857,0.292795,-0.386532,0.043952,-0.056640,0.147759,-0.027741,-0.023302,0.033641,-0.080011,0.191734,0.832880,0.387101,-0.304152,0.006003,-0.150323,0.050006,0.276766,-0.101714,0.083234,0.011099,-0.064434,-0.074423,-0.027453,0.005086,-0.036148,0.126355,-0.043163,-0.062473,-0.049253,-0.048674,-0.036397,0.034332,0.026449,0.071324,-0.069377,-0.026302,-0.012400,-0.100806,-0.079706,-0.056599,0.005431,0.039385,0.018601,-0.054873,-0.019722,-0.018006,-0.047266,0.056301,-0.010723,-0.096755,0.094104,-0.027149,0.087602,-0.045213,-0.026556,0.021128,0.208846,-0.026770,-0.006963,-0.083541,0.050173,0.148456,-0.144775,0.189205,-0.013399,-0.091328,-0.178849,-0.169060,0.144736,0.007208,-0.122847,-0.159956,0.093264,-0.114599,0.119554,-0.097462,0.015153,0.000403,0.158651,-0.110623,-0.017557,0.089849,0.027461,0.051126,0.075883,-0.032646,-0.025751,-0.052141,-0.014147,0.035829,0.072213,-0.004349,-0.113046,0.082648,0.012799,-0.018040,-0.087561,0.031641,0.009772,0.075496,0.007306,0.031972,-0.017281,0.002332,0.069116
1,0.632527,-0.513145,-0.159173,0.113324,0.812374,0.205336,0.577209,-0.559892,-0.033838,-0.298521,0.300487,-0.399187,0.078961,-0.134058,-0.065143,0.437688,-0.447482,0.254005,-0.008192,-0.390920,-0.095669,0.039855,-0.105549,0.327949,0.340830,-0.395168,0.021087,-0.022016,0.125976,-0.006007,-0.031940,0.021036,-0.065959,0.183012,0.819589,0.378888,-0.279830,0.008563,-0.146340,0.057366,0.262930,-0.075888,0.077489,0.005268,-0.070897,-0.063551,-0.043351,0.016535,-0.041044,0.115955,-0.043309,-0.053285,-0.051531,-0.075798,-0.031231,0.021236,0.017699,0.078178,-0.060916,-0.016829,-0.021215,-0.101698,-0.073307,-0.054853,0.012684,0.047118,0.020248,-0.048795,-0.023072,-0.012513,-0.043680,0.052881,-0.017655,-0.107210,0.099398,-0.029331,0.087049,-0.041116,-0.026695,0.017696,0.208466,-0.025762,-0.001036,-0.085761,0.046122,0.160619,-0.139709,0.179108,-0.017610,-0.090471,-0.168175,-0.179559,0.148830,0.013586,-0.130081,-0.156759,0.089466,-0.120351,0.116721,-0.097065,0.018440,-0.007684,0.157432,-0.104702,-0.020342,0.089730,0.030448,0.051572,0.079501,-0.033039,-0.034556,-0.053188,-0.010032,0.033904,0.076178,-0.009262,-0.110854,0.080540,0.008234,-0.020497,-0.090566,0.027976,0.007454,0.072534,0.000071,0.034379,-0.014078,-0.000496,0.068476
2,-0.689568,-0.672794,-0.058345,0.166593,-0.497949,1.513468,-0.124929,0.027424,-0.492693,-0.26683

In [5]:
# the calculation of the similarities between the articles
article_similarities = cosine_similarity(articles_profiles ,articles_profiles )

article_id2idx, article_idx2id = {}, {}
for idx, article_id in enumerate(articles['article_id']):
    article_id2idx[article_id] = idx
    article_idx2id[idx] = article_id
    
def get_kitems_item_recommendations(article_id, topk=12, factor=2, remove_input_article=True):
    
    # Given the article id, get the correct index in the matrix
    article_idx = article_id2idx[int(article_id)]
    
    # Get the topk most similar articles (identified by their indices)
    most_similar_article_indices = np.argsort(article_similarities[article_idx])[::-1][:topk*factor]
    most_similar_article_scores = article_similarities[article_idx][most_similar_article_indices]
    
    # Remove the input article from the list of most similar articles (optional)
    if remove_input_article == True:
        # Get the index of input article id
        idx = np.where(most_similar_article_indices == article_idx)[0]
        # Remove article id from list of most similar article
        most_similar_article_indices = np.delete(most_similar_article_indices, idx)
        most_similar_article_scores = np.delete(most_similar_article_scores, idx)
    
    # Get the ids of the articles from the indicies of the articles
    recommended_article_ids = [ article_idx2id[idx] for idx in most_similar_article_indices]
    
    # Return the recommendations as ordered list of article ids
    return recommended_article_ids, most_similar_article_scores

article_list = list(pd.Series(action_test["article_id"]).unique())
article_recom_id = {}
article_recom_score = {}

for article_id in article_list:
    recommended_article_ids, recommended_article_scores = get_kitems_item_recommendations(article_id)
    article_recom_id[article_id] = recommended_article_ids
    article_recom_score[article_id] = recommended_article_scores

In [18]:
recommended_article_ids = get_kitems_item_recommendations(108775044)
for rank, article_id in enumerate(recommended_article_ids):
    print('[Rank {}] {}'.format(rank+1, int(article_id) ))

[Rank 1] 110065002
[Rank 2] 111586001
[Rank 3] 114428030
[Rank 4] 111565001
[Rank 5] 111565003
[Rank 6] 118458004
[Rank 7] 144993001
[Rank 8] 126589010
[Rank 9] 129085001
[Rank 10] 111593001
[Rank 11] 120129014
[Rank 12] 145872001


### 4.2 Method 2: Item Similarity Recommender
A model that ranks an item according to its similarity to other items observed for the user in question. we implement it by leveraging `turicreate` package.

In [69]:
def get_kitems_cf_recommendations(train_data, topk, user_id, item_id, target, users_to_recommend):

    model = tc.item_similarity_recommender.create(train_data, 
                                                      user_id=user_id, 
                                                      item_id=item_id, 
                                                      target=target,
                                                      similarity_type='cosine', verbose=False)
        
    recom = model.recommend(users=users_to_recommend, k=topk, verbose=False)
    return model, recom


users_to_recommend = list(pd.Series(action_test["customer_id"]).unique())
m2_model, m2_recomms= get_kitems_cf_recommendations(action_train, 12, user_id, item_id, target, users_to_recommend=users_to_recommend )

In [70]:
m2_recomms

customer_id,article_id,score,rank
37407537894153080,699003002,0.12777531147003174,1
37407537894153080,147339034,0.12777531147003174,2
37407537894153080,485591004,0.12777531147003174,3
37407537894153080,797594002,0.12777531147003174,4
37407537894153080,761634003,0.12777531147003174,5
37407537894153080,633825001,0.12777531147003174,6
37407537894153080,646520002,0.12777531147003174,7
37407537894153080,704658001,0.12777531147003174,8
37407537894153080,801252002,0.12777531147003174,9
37407537894153080,708419001,0.12777531147003174,10


### 4.3 Method 3: Collaborative Filtering (CF) - Factorization Recommender
A FactorizationRecommender learns latent factors for each user and item and uses them to make rating predictions. we implement it by leveraging `turicreate` package.

In [76]:
def get_kitems_cf_recommendations(train_data, topk, user_id, item_id,user_data,item_data, target, users_to_recommend):
    model = tc.recommender.create(train_data, 
                                 user_id=user_id, 
                                 item_id=item_id, 
                                 user_data=user_data,
                                 item_data=item_data,
                                 target=target,
                                 verbose=False)
        
    recom = model.recommend(users=users_to_recommend, k=topk, verbose=False)
    
    return model, recom


user_data = tc.SFrame(customers)
item_data = tc.SFrame(articles)
m3_model, m3_recomms = get_kitems_cf_recommendations(action_train, 12, user_id, item_id, user_data, item_data, target, users_to_recommend=users_to_recommend )

Preparing data set.

Data has 133570 observations with 12610 users and 51827 items.

Data prepared in: 3.32081s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.518578

Final training RMSE: 0.514949

In [78]:
m3_recomms

customer_id,article_id,score,rank
37407537894153080,156231001,1.1705443048080055,1
37407537894153080,706016001,1.1678876983056719,2
37407537894153080,228257001,1.1674190188055757,3
37407537894153080,610776002,1.1664422142535957,4
37407537894153080,562245001,1.1662219375143048,5
37407537894153080,158340001,1.1655752499863172,6
37407537894153080,436261001,1.165473469516652,7
37407537894153080,448509014,1.165453162854266,8
37407537894153080,806605001,1.165307490456695,9
37407537894153080,561445005,1.1652520269656546,10


## 5. Model Evaluation
### 5.1 Evaluation Metrics - Average Precision at K

In [ ]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])


### 5.2 Method 1: Content-based similiartiy Evaluation

In [65]:
user_list =  list(pd.Series(action_test["customer_id"]).unique())
topk = 12
factor = 2
recommened_user_items = []
true_user_items = []

for user in user_list:
    buy_record = list(action_test_df.loc[(action_test_df.customer_id == user), 'article_id'])
    recom_ids = list()
    recom_scores = list()
    for buy in buy_record:
        recom_ids = recom_ids + list(article_recom_id[buy])
        recom_scores = recom_scores + list(article_recom_score[buy])
    selected_idxs = np.argsort(np.array(recom_scores))[::-1][:topk]
    
    # Pick a random topk sample of topk*factor most similar articles
    selected_idxs = np.random.choice(selected_idxs, size=topk, replace=False)
    recom_ids = np.array(recom_ids)
    recommended_article_ids = recom_ids[selected_idxs]
    #a = list(pd.Series(recommended_article_ids).astype('str').str.slice(0,6))

    recommened_user_items.append(list(pd.Series(recommended_article_ids).astype('str').str.slice(0,6)))
    true_user_items.append(list(pd.Series(buy_record).astype('str').str.slice(0,6)))
    

In [68]:
m1_results

,customer_id,predicted_y,actual_y
0,37407537894153080,"[704570, 884391, 667036, 454804, 591702, 74630...","[746300, 840210]"
1,45567281885624136,"[638416, 654423, 723469, 614496, 650279, 52900...",[723469]
2,53071904810868432,"[854356, 868258, 820308, 684087, 865879, 60930...",[490436]
3,69791325512213248,"[716672, 712216, 712216, 712216, 887770, 76218...","[821163, 896527]"
4,69968312338776440,"[399223, 399223, 914441, 914441, 914441, 39922...","[399223, 682261]"
...,...,...,...
689,9146937598958496768,"[884174, 870434, 884174, 632307, 874224, 63230...",[632307]
690,9164779220409513984,"[750422, 865076, 610776, 831412, 690933, 69093...","[407354, 519079, 570004, 610776, 750422, 77521..."
691,9186131866243185664,"[622958, 616711, 660599, 564786, 564786, 74292...","[297067, 380193, 437129, 525335, 539448, 54191..."
692,9189349431878553600,"[798579, 872266, 872261, 607310, 864216, 79857...",[886189]


In [67]:
m1_results = pd.DataFrame({"customer_id": user_list, 
                           "predicted_y": recommened_user_items,
                          "actual_y": true_user_items})
m1_mapk = mapk(m1_results["actual_y"], m1_results["predicted_y"], k=12)
print("The average precision of M1 method is : {}".format(m1_mapk))

The average precision of M1 method is : 0.20289004543875455


### 5.3 Method 2: Item Similarity Recommender Evaluation

In [71]:
# M2 predicted Values
m2_results = pd.DataFrame(m2_recomms.copy())
m2_results['article_id'] = m2_results['article_id'].astype('str')
m2_results['article_id'] = m2_results['article_id'].str.slice(0,6)
m2_results = m2_results.groupby('customer_id')['article_id'].apply(list).reset_index()
m2_results

,customer_id,article_id
0,37407537894153080,"[699003, 147339, 485591, 797594, 761634, 63382..."
1,45567281885624136,"[882925, 816363, 811953, 807748, 408875, 61008..."
2,53071904810868432,"[643202, 777059, 877069, 556539, 645895, 86547..."
3,69791325512213248,"[872000, 722169, 773757, 869085, 862399, 84711..."
4,69968312338776440,"[896319, 774918, 774918, 795458, 810172, 81171..."
...,...,...
689,9146937598958496768,"[860322, 742429, 736489, 576773, 763285, 87726..."
690,9164779220409513984,"[640108, 617828, 698382, 725473, 797872, 91943..."
691,9186131866243185664,"[696511, 591333, 612133, 587278, 557241, 53929..."
692,9189349431878553600,"[700931, 754242, 690541, 854920, 724904, 77454..."


In [49]:
# Real Values
test_results = pd.DataFrame(action_test)
test_results['article_id'] = test_results['article_id'].astype('str')
test_results['article_id'] = test_results['article_id'].str.slice(0,6)
test_results = test_results.groupby('customer_id')['article_id'].apply(list).reset_index()
test_results

,customer_id,article_id
0,37407537894153080,"[746300, 840210]"
1,45567281885624136,[723469]
2,53071904810868432,[490436]
3,69791325512213248,"[821163, 896527]"
4,69968312338776440,"[399223, 682261]"
...,...,...
689,9146937598958496768,[632307]
690,9164779220409513984,"[407354, 519079, 570004, 610776, 750422, 77521..."
691,9186131866243185664,"[297067, 380193, 437129, 525335, 539448, 54191..."
692,9189349431878553600,[886189]


In [72]:
m2_results = m2_results.merge(test_results, how='left', on='customer_id')
m2_results

,customer_id,article_id_x,article_id_y
0,37407537894153080,"[699003, 147339, 485591, 797594, 761634, 63382...","[746300, 840210]"
1,45567281885624136,"[882925, 816363, 811953, 807748, 408875, 61008...",[723469]
2,53071904810868432,"[643202, 777059, 877069, 556539, 645895, 86547...",[490436]
3,69791325512213248,"[872000, 722169, 773757, 869085, 862399, 84711...","[821163, 896527]"
4,69968312338776440,"[896319, 774918, 774918, 795458, 810172, 81171...","[399223, 682261]"
...,...,...,...
689,9146937598958496768,"[860322, 742429, 736489, 576773, 763285, 87726...",[632307]
690,9164779220409513984,"[640108, 617828, 698382, 725473, 797872, 91943...","[407354, 519079, 570004, 610776, 750422, 77521..."
691,9186131866243185664,"[696511, 591333, 612133, 587278, 557241, 53929...","[297067, 380193, 437129, 525335, 539448, 54191..."
692,9189349431878553600,"[700931, 754242, 690541, 854920, 724904, 77454...",[886189]


In [81]:
m2_mapk = mapk(m2_results["article_id_y"], m2_results["article_id_x"], k=12)
print("The average precision of M2 method is : {}".format(m2_mapk))

The average precision of M2 method is : 0.012504587351489368


### 5.4 Method 3: Factorization Recommender Evaluation

In [77]:
# M3 predicted Values
m3_results = pd.DataFrame(m3_recomms.copy())
m3_results['article_id'] = m3_results['article_id'].astype('str')
m3_results['article_id'] = m3_results['article_id'].str.slice(0,6)
m3_results = m3_results.groupby('customer_id')['article_id'].apply(list).reset_index()
m3_results

,customer_id,article_id
0,37407537894153080,"[156231, 706016, 228257, 610776, 562245, 15834..."
1,45567281885624136,"[156231, 706016, 228257, 610776, 562245, 56144..."
2,53071904810868432,"[156231, 706016, 228257, 610776, 562245, 15834..."
3,69791325512213248,"[156231, 706016, 228257, 610776, 562245, 15834..."
4,69968312338776440,"[156231, 706016, 228257, 610776, 562245, 56144..."
...,...,...
689,9146937598958496768,"[156231, 706016, 228257, 610776, 562245, 15834..."
690,9164779220409513984,"[156231, 706016, 228257, 610776, 562245, 15834..."
691,9186131866243185664,"[156231, 706016, 228257, 610776, 562245, 56144..."
692,9189349431878553600,"[156231, 706016, 228257, 610776, 562245, 15834..."


In [79]:
m3_results = m3_results.merge(test_results, how='left', on='customer_id')
m3_results

,customer_id,article_id_x,article_id_y
0,37407537894153080,"[156231, 706016, 228257, 610776, 562245, 15834...","[746300, 840210]"
1,45567281885624136,"[156231, 706016, 228257, 610776, 562245, 56144...",[723469]
2,53071904810868432,"[156231, 706016, 228257, 610776, 562245, 15834...",[490436]
3,69791325512213248,"[156231, 706016, 228257, 610776, 562245, 15834...","[821163, 896527]"
4,69968312338776440,"[156231, 706016, 228257, 610776, 562245, 56144...","[399223, 682261]"
...,...,...,...
689,9146937598958496768,"[156231, 706016, 228257, 610776, 562245, 15834...",[632307]
690,9164779220409513984,"[156231, 706016, 228257, 610776, 562245, 15834...","[407354, 519079, 570004, 610776, 750422, 77521..."
691,9186131866243185664,"[156231, 706016, 228257, 610776, 562245, 56144...","[297067, 380193, 437129, 525335, 539448, 54191..."
692,9189349431878553600,"[156231, 706016, 228257, 610776, 562245, 15834...",[886189]


In [82]:
m3_mapk = mapk(m3_results["article_id_y"], m3_results["article_id_x"], k=12)
print("The average precision of M3 method is : {}".format(m3_mapk))

The average precision of M3 method is : 0.005802224246028282
